# IMPORTS

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv(r"BTC-Dataset-USDM-2024-04-01 - 2024-04-17 - 5M.csv")

# Define Price Based Indicators

In [3]:
def calculate_ema(data, periods=[12, 26,50,100,200,500]):
    for period in periods:
        ema_name = f'EMA_{period}'
        data[ema_name] = data['close'].ewm(span=period, adjust=False).mean()
    return data


def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def calculate_stochastic(data, k_period=14, d_period=3):
    lowest_low = data['low'].rolling(window=k_period).min()
    highest_high = data['high'].rolling(window=k_period).max()
    data['stochastic_%K'] = ((data['close'] - lowest_low) / (highest_high - lowest_low)) * 100
    data['stochastic_%D'] = data['stochastic_%K'].rolling(window=d_period).mean()
    return data


# Add The Indicators to Dataframe

In [4]:
# Calculate RSI
df['RSI'] = calculate_rsi(df['close'])

# Calculate Stochastic Oscillator
df = calculate_stochastic(df)

# # Calculate EMAs
df = calculate_ema(df)


# Clean The Dataframe & Remove Unneccassary Columns

In [5]:
df['close_pct'] = df['close'].pct_change() * 100
df = df.drop(columns=['Unnamed: 0', 'side', 'high','low','close','open','taker_buy_volume','last_fill_volume', 'accumulated_fill_volume','volume','taker_buy_quote_volume'])
df.dropna(inplace=True)
df.set_index('date_time', inplace=True)

In [6]:
df

,quote_volume,count,volume_true,volume_false,vdiff(false-true),original_volume,RSI,stochastic_%K,stochastic_%D,EMA_12,EMA_26,EMA_50,EMA_100,EMA_200,EMA_500,close_pct
date_time,,,,,,,,,,,,,,,,
2024-04-01 01:15:00,2.564227e+07,7159,1.884207e+07,6.800199e+06,-1.204187e+07,0.0,35.349695,29.321845,43.700747,71156.229534,71207.428889,71249.935060,71282.291062,71301.714373,71314.576819,-0.140566
2024-04-01 01:20:00,3.417142e+07,7494,7.795057e+06,2.637636e+07,1.858131e+07,0.0,43.569315,51.650541,43.937851,71150.194221,71200.730453,71244.721920,71279.017972,71299.876419,71313.788089,0.106980
2024-04-01 01:25:00,2.864201e+07,7649,1.328454e+07,1.535747e+07,2.072927e+06,0.0,35.545385,24.416619,35.129668,71130.364341,71187.439308,71235.960276,71273.914646,71297.104514,71312.620472,-0.134567
2024-04-01 01:30:00,3.601778e+07,7971,1.419771e+07,2.182029e+07,7.622582e+06,0.0,44.933878,66.278027,47.448396,71134.492904,71185.199359,71232.871638,71271.603465,71295.712429,71312.000031,0.191351
2024-04-01 01:35:00,4.994263e+07,12218,3.093844e+07,1.900398e+07,-1.193446e+07,27715458.4,40.161655,2.864583,31.186410,71102.155534,71165.873481,71220.770789,71264.726168,71292.016783,71310.452326,-0.327303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-17 23:05:00,3.966970e+07,10868,1.751363e+07,2.215607e+07,4.642434e+06,6319144.8,73.416518,88.980399,87.237756,61503.075030,61392.921595,61311.984491,61403.070173,61830.899388,62625.473167,0.161260
2024-04-17 23:10:00,3.071231e+07,8083,2.352097e+07,7.216019e+06,-1.630495e+07,0.0,70.252445,68.769560,78.724669,61510.925025,61404.860736,61321.479217,61406.060863,61828.145165,62621.196228,-0.198940
2024-04-17 23:15:00,5.202591e+07,13121,3.800741e+07,1.399381e+07,-2.401360e+07,0.0,54.311572,34.314067,64.021342,61488.690406,61402.011793,61323.240816,61405.275499,61823.550686,62616.187062,-0.304935


In [7]:
df.to_csv('BTC Aprill-Indicators-5M.csv')